In [1]:
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import sqlite3

In [23]:
# A function to to connect and pull data from the metatrader apr
def get_data(symbol):
    # Getting data on the 1 hour timeframe
    if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()
    # set the symbol and timefram
    timeframe = mt5.TIMEFRAME_M15   # for one-hour bars
    #dates for retrieving the history
    date_from = datetime(2007, 12, 1)
    today = datetime.today()
    # get the history
    history = mt5.copy_rates_range(symbol, timeframe, date_from, today)
    if history is not None and len(history) > 0:
        # create DataFrame out of the obtained data
        rates_frame = pd.DataFrame(history).drop(['spread','real_volume'], axis =1)
        # convert time in seconds into the datetime format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    else:
        print("No data for the requested period")
    # terminate the connection to the MetaTrader 5 terminal
    mt5.shutdown()
    data = rates_frame

    return data

data = get_data('XAUUSDm')
data

,time,open,high,low,close,tick_volume
0,2020-08-21 15:30:00,1942.972,1943.144,1935.649,1937.479,2266
1,2020-08-21 15:45:00,1937.587,1940.241,1936.407,1939.244,1953
2,2020-08-21 16:00:00,1939.274,1939.274,1933.436,1937.451,2461
3,2020-08-21 16:15:00,1937.453,1938.081,1933.428,1935.001,1782
4,2020-08-21 16:30:00,1934.988,1934.988,1932.031,1934.384,1983
...,...,...,...,...,...,...
100634,2024-11-22 20:45:00,2707.073,2707.137,2705.283,2705.841,1625
100635,2024-11-22 21:00:00,2705.806,2709.150,2705.707,2708.792,1145
100636,2024-11-22 21:15:00,2708.824,2709.859,2708.435,2709.157,983
100637,2024-11-22 21:30:00,2709.178,2712.027,2707.843,2711.936,1483


In [4]:
data.to_csv('price_data')

In [14]:
def get_fred_data(api_key, series_id, start_date="2000-01-01"):
    """
    Fetches monthly economic data from FRED.
    
    Parameters:
        api_key (str): Your FRED API key.
        series_id (str): The FRED series ID for the desired economic indicator.
        start_date (str): The start date for data in "YYYY-MM-DD" format.
        
    Returns:
        pd.DataFrame: A DataFrame with the date as the index and the series data.
    """
    url = f"https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "frequency": "m",  # monthly frequency
        "observation_start": start_date
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if "observations" not in data:
        print(f"Error fetching data for series {series_id}.")
        return pd.DataFrame()
    
    # Convert the observations into a DataFrame
    df = pd.DataFrame(data["observations"])
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index("date")
    df = df[["value"]]
    df.columns = [series_id]
    df[series_id] = pd.to_numeric(df[series_id], errors='coerce')
    
    # Add a "previous" column by shifting the actual values
    df[f"{series_id}_previous"] = df[series_id].shift(1)
    
    return df
    
# Define your FRED API key
fred_api_key = "7580ed8bbc13d1a99d0bc31da5870a8d"

# Fetch data for Unemployment rate and NFP (Non-Farm Payroll)
nfp_data = get_fred_data(fred_api_key, series_id="PAYEMS")
unemployment_data = get_fred_data(fred_api_key, series_id="UNRATE")

***Data from forexfactory***

In [13]:
# non_farm_payroll = pd.read_csv('non_farm_payroll.csv')
non_farm_payroll = pd.read_csv('non_farm_payroll.csv', encoding='iso-8859-1')
unempployment_rate = pd.read_csv('US_unemployment_rate.csv', encoding='iso-8859-1')
non_farm_payroll

,Expected Impact / Date,Actual,Forecast,Previous
0,"Nov 1, 2024",12K,106K,223K
1,"Oct 4, 2024",254K,147K,159K
2,"Sep 6, 2024",142K,164K,89K
3,"Aug 2, 2024",114K,176K,179K
4,"Jul 5, 2024",206K,191K,218K
...,...,...,...,...
210,"May 4, 2007",88K,107K,177K
211,"Apr 6, 2007",180K,135K,113K
212,"Mar 9, 2007",97K,100K,146K
213,"Feb 2, 2007",111K,146K,206K


In [14]:
unempployment_rate

,Expected Impact / Date,Actual,Forecast,Previous
0,"Nov 1, 2024",4.10%,4.10%,4.10%
1,"Oct 4, 2024",4.10%,4.20%,4.20%
2,"Sep 6, 2024",4.20%,4.20%,4.30%
3,"Aug 2, 2024",4.30%,4.10%,4.10%
4,"Jul 5, 2024",4.10%,4.00%,4.00%
...,...,...,...,...
210,"May 4, 2007",4.50%,4.50%,4.40%
211,"Apr 6, 2007",4.40%,4.60%,4.50%
212,"Mar 9, 2007",4.50%,4.60%,4.60%
213,"Feb 2, 2007",4.60%,4.50%,4.50%
